In [1]:
import numpy as np
from sega_learn.neural_networks.neuralNetwork import NeuralNetwork

In [2]:
# Sample data
X_train = np.random.randn(100, 10)
y_train = np.random.randint(0, 2, 100)
# One-hot encoding
y_train_ohe = np.eye(2)[y_train]

X_val = np.random.randn(20, 10)
y_val = np.random.randint(0, 2, 20)

In [3]:
# Neural network configurations
layer_sizes = [10, 5, 3, 2]
dropout_rate = 0
reg_lambda = 0
activations = ['relu', 'relu', 'relu']

In [4]:
# Initialize neural networks
nn_numba = NeuralNetwork(layer_sizes, dropout_rate, reg_lambda, activations, use_numba=True, compile_numba=False)
nn_no_numba = NeuralNetwork(layer_sizes, dropout_rate, reg_lambda, activations, use_numba=False)

In [5]:
# Models are initialized with the same parameters but receive different randomized weights and biases

# Set weights to the same values for comparison
for i in range(len(nn_numba.weights)):
    nn_numba.weights[i] = np.random.randn(*nn_numba.weights[i].shape)
    nn_no_numba.weights[i] = nn_numba.weights[i].copy()
    
# Set biases to the same values for comparison
for i in range(len(nn_numba.biases)):
    nn_numba.biases[i] = np.random.randn(*nn_numba.biases[i].shape)
    nn_no_numba.biases[i] = nn_numba.biases[i].copy()

# Assert equality of weights
for w1, w2 in zip(nn_numba.weights, nn_no_numba.weights):
    assert np.array_equal(w1, w2), "Weights are not equal!"
print("Weights are equal!")

# Assert equality of biases
for b1, b2 in zip(nn_numba.biases, nn_no_numba.biases):
    assert np.array_equal(b1, b2), "Biases are not equal!"
print("Biases are equal!")

Weights are equal!
Biases are equal!


In [6]:
# Model Layers are initialized with the same parameters but receive different randomized weights and biases

# Set weights and biases to the same values for comparison
for i in range(len(nn_numba.layers)):
    nn_numba.layers[i].weights = np.random.randn(*nn_numba.layers[i].weights.shape)
    nn_no_numba.layers[i].weights = nn_numba.layers[i].weights.copy()
    
    nn_numba.layers[i].biases = np.random.randn(*nn_numba.layers[i].biases.shape)
    nn_no_numba.layers[i].biases = nn_numba.layers[i].biases.copy()

# Assert equality of weights
for l1, l2 in zip(nn_numba.layers, nn_no_numba.layers):
    assert np.array_equal(l1.weights, l2.weights), "Layer weights are not equal!"
print("Layer weights are equal!")

# Assert equality of biases
for l1, l2 in zip(nn_numba.layers, nn_no_numba.layers):
    assert np.array_equal(l1.biases, l2.biases), "Layer biases are not equal!"
print("Layer biases are equal!")

Layer weights are equal!
Layer biases are equal!


In [7]:
# Forward pass comparison
output_numba = nn_numba.forward(X_train)
output_no_numba = nn_no_numba.forward(X_train)

tolerance = 1e-2
if np.allclose(output_numba, output_no_numba, atol=tolerance):
    print(f"Forward pass outputs are equal within tolerance of {tolerance}.")
else:
    print(f"Forward pass outputs are not equal within tolerance of {tolerance}.")
    print(f"First 5 outputs (Numba): \n{output_numba[:5]}")
    print(f"First 5 outputs (No Numba): \n{output_no_numba[:5]}")

Forward pass outputs are equal within tolerance of 0.01.


In [8]:
# Backward pass comparison

# Zero gradients for both models
for i, layer in enumerate(nn_numba.layers):
    layer.zero_grad()
    nn_numba.dWs_cache[i].fill(0)
    nn_numba.dbs_cache[i].fill(0)
for i, layer in enumerate(nn_no_numba.layers):
    layer.zero_grad()

nn_numba.backward(y_train)
nn_no_numba.backward(y_train)

# Layers gradients comparison
# Numba stores in seperate weight_gradients and bias_gradients
# Vanilla stores in gradients as a list of tuples (dW, db)
# Compare gradients

tolerance = 1e-2

print("\nComparing gradients:")
print("-"*75, end="")
for i in range(len(nn_numba.layers)):
    weights_close = np.allclose(nn_numba.layers[i].weight_gradients, nn_no_numba.layers[i].gradients[0], atol=tolerance)
    if not weights_close:
        print(f"\nLayer {i} weight gradients are not equal within tolerance of {tolerance}.")
        # Find index of first non-equal element
        diff_index = np.where(nn_numba.layers[i].weight_gradients != nn_no_numba.layers[i].gradients[0])[0][0]
        print(f"First non-equal element at index {diff_index}:")
        print(f"Numba   {diff_index}: {nn_numba.layers[i].weight_gradients[diff_index]}")
        print(f"Vanilla {diff_index}: {nn_no_numba.layers[i].gradients[0][diff_index]}")
        
    else:
        print(f"\nLayer {i} weight gradients are equal within tolerance of {tolerance}.")

print("\nComparing biases:")
print("-"*75, end="")
for i in range(len(nn_numba.layers)):
    biases_close = np.allclose(nn_numba.layers[i].bias_gradients, nn_no_numba.layers[i].gradients[1], atol=tolerance)
    if not biases_close:
        print(f"\nLayer {i} bias gradients are not equal within tolerance of {tolerance}.")
        # Find index of first non-equal element
        diff_index = np.where(nn_numba.layers[i].bias_gradients != nn_no_numba.layers[i].gradients[1])[0][0]
        print(f"First non-equal element at index {diff_index}:")
        print(f"Numba   {diff_index}: {nn_numba.layers[i].bias_gradients[diff_index]}")
        print(f"Vanilla {diff_index}: {nn_no_numba.layers[i].gradients[1][diff_index]}")
        
    else:
        print(f"\nLayer {i} bias gradients are equal within tolerance of {tolerance}.")
        


Comparing gradients:
---------------------------------------------------------------------------
Layer 0 weight gradients are not equal within tolerance of 0.01.
First non-equal element at index 0:
Numba   0: [ 0.0434958   0.00952862 -0.03480999 -0.00217476 -0.00431811]
Vanilla 0: [ 0.08096992 -0.02321791  0.00917463 -0.02276502  0.02424956]

Layer 1 weight gradients are not equal within tolerance of 0.01.
First non-equal element at index 0:
Numba   0: [ 0.4040638  -0.08350137  0.20198455]
Vanilla 0: [0.74994496 0.5420762  0.4779987 ]

Layer 2 weight gradients are equal within tolerance of 0.01.

Comparing biases:
---------------------------------------------------------------------------
Layer 0 bias gradients are not equal within tolerance of 0.01.
First non-equal element at index 0:
Numba   0: [ 0.20049063  0.07895708 -0.09315961  0.01468131 -0.05082866]
Vanilla 0: [ 0.2780606  -0.06528172  0.06557862  0.15358646  0.29802422]

Layer 1 bias gradients are not equal within tolerance o

In [9]:
# Training comparison
nn_numba.train(X_train, y_train, X_val, y_val, epochs=1, batch_size=10, use_tqdm=False)
nn_no_numba.train(X_train, y_train, X_val, y_val, epochs=1, batch_size=10, use_tqdm=False)

# Get losses
loss_numba = nn_numba.calculate_loss(X_val, y_val)
loss_no_numba = nn_no_numba.calculate_loss(X_val, y_val)

tolerance = 1e-3
print(f"Training losses are equal to within tolerance of {tolerance}: {np.allclose(loss_numba, loss_no_numba, atol=tolerance)}")
print(f"\tNumba loss:   {loss_numba}\n\tVanilla loss: {loss_no_numba}")

Epoch 1/1 - Loss: 0.7659, Acc: 0.5400, Val Loss: 0.7415, Val Acc: 0.5500
Epoch 1/1 - Loss: 0.7658, Acc: 0.5400, Val Loss: 0.7414, Val Acc: 0.5500
Training losses are equal to within tolerance of 0.001: True
	Numba loss:   0.7415464788683098
	Vanilla loss: 0.7414206072942819


In [10]:
# Evaluation comparison
accuracy_numba, _ = nn_numba.evaluate(X_val, y_val)
accuracy_no_numba, _ = nn_no_numba.evaluate(X_val, y_val)

tolerance = 1e-3
print(f"Evaluation accuracies are equal to within tolerance of {tolerance}: {np.allclose(accuracy_numba, accuracy_no_numba, atol=tolerance)}")
print(f"\tNumba accuracy:   {accuracy_numba}\n\tVanilla accuracy: {accuracy_no_numba}")


Evaluation accuracies are equal to within tolerance of 0.001: True
	Numba accuracy:   0.55
	Vanilla accuracy: 0.55
